<style>
@font-face {font-family: "B Nazanin"; src: url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.eot"); src: url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.eot?#iefix") format("embedded-opentype"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.woff2") format("woff2"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.woff") format("woff"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.ttf") format("truetype"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.svg#B Nazanin") format("svg"); }
    </style>
<div dir="rtl" align="center" style ='font-family: "B Nazanin";'>
    <h1>
        تمرین سوم درس پردازش زبان‌های طبیعی
    </h1>
    <h3>
        گردآورندگان:<br/>
        ساحل مس‌فروش، سروش تابش، درنا دهقانی
    </h3>
    
</div>

<br/>
<br/>


<div dir="rtl"  style ='font-family: "B Nazanin";'>
    ما در این تمرین ترک "تکمیل مصراع دوم با رعایت وزن شعر" را انتخاب کردیم. در این تمرین قصد داریم به کمک مدل‌های زبانی مناسب، چند کلمه به عنوان یک مصرع از بیت را به عنوان ورودی بگیریم و چند کلمه به عنوان مصرع دوم این بیت را خروجی دهیم. در این تمرین ما از وبسایت گنجور برای جمع‌آوری داده کمک گرفتیم و از چندین قالب شعری استفاده کردیم، زیرا قافیه داشتن یا نداشتن به عنوان ورودی داده می‌شود. اگر نیاز به هم‌قافیه بودن دو مصراع بود، از قالب مثنوی نظیر شاهنامه‌ی فردوسی کمک می‌گیریم و اگر نیاز به هم‌قافیه بودن دو مصراع نبود می‌توانیم از قالب غزل نظیر اشعار حافظ استفاده کنیم. <br>
    برای ساخت چنین سیستمی از چند مدل زبانی استفاده می‌کنیم: مدل n-gram، مدل encoder-decoder با یک شبکه LSTM به عنوان encoder، و مدلی پیرو مکانیسم توجه.
</div>

In [ ]:
! pip install pandas
! pip install hazm
! pip install transformers
! pip install torch
! pip install nltk
! pip install numpy
! pip install codec
! pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 27.4 MB/s 
     |████████████████████████████████| 1.4 MB 49.5 MB/s 
     |████████████████████████████████| 233 kB 33.7 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394484 sha256=34707869264d8f0328969326dcea1c27faa88e32756fed05cade6d7f71b570fb
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=153864 sha256=dabcc24e75f8d4a3402677796a9a009da50d4cccbba069c7d35ae456edaa21b0
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 26.2 MB/s 
     |████████████████████████████████| 6.6 MB 41.0 MB/s 
     |████████████████████████████████| 596 kB 54.6 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement codec (from versions: none)
ERROR: No matching distribution fou

In [ ]:
import random 
import numpy as np
import nltk
import pandas as pd
import codecs
import tqdm
import os
import re
import glob
from __future__ import unicode_literals
from hazm import *
from nltk import FreqDist
import itertools
import pandas as pd
import torch
import wandb

<div dir="rtl"  style ='font-family: "B Nazanin";'>
    <h3> 1. دوباره‌نویسی داده </h3>
    از آنجایی که در داده‌ی گرفته شده، ابتدا و انتهای ابیات و مصراع‌ها مشخص نبود، به کمک کد زیر این داده را بازنویسی کردیم. ابتدای هر مصرع جدید با __BOM__ و پایان بیت با __EOM__ مشخص گردید.</div>

In [ ]:
path = 'all_norm.txt'
beyt_file = ''
with open(path, 'r', encoding="utf-8") as fp:
    lines = fp.readlines()
    for i in range(0, len(lines) - 1, 2):
        mesra1 = '__BOM__ ' + lines[i].strip()
        mesra2 = '__BOM__ ' + lines[i + 1].strip()
        b = mesra1.strip() + ' ' + mesra2.strip() + ' __EOM__\n'
        beyt_file += b
        
with open('all_beyt.txt', 'w', encoding="utf-8") as fp:
    fp.write(beyt_file)

<div dir="rtl"  style ='font-family: "B Nazanin";'>
    <h3> 2. مدل n-gram </h3>
    برای ساخت این مدل از کلاس Ngram استفاده کردیم که به کمک مطالب گفته شده در کلاس نوشته شده است. <br> در این مدل، دیتای لازم برای train، مقدار مورد استفاده برای laplace smoothing و n به عنوان ورودی گرفته می‌شوند و بر اساس آنها مدل ساخته می‌شود. هنگام train کردن، اگر با token مواجه شدیم که تکرار آن در کل دیتا بیش از 1 نبود از عبارت __UNK__ به جای آن استفاده می‌کنیم. سپس احتمال هر n-gram متناسب با تعداد آن در کل و تعداد (n_1)-gram ها محاسبه شده و laplace smoothing روی آن انجام می‌گیرد. نهایتاً n-gramها با توجه به احتمالشان به طور کاهشی مرتب می‌شوند، تا سرعت یافتن مصراع دوم بالاتر رود.<br> سپس یک مصراع به عنوان ورودی گرفته می‌شود. با توجه به n، n_1 token پایانی مصراع ورودی پیش پردازش می‌شود و بین n-gramهای موجود که شروعشان با آن n_1 token باشد، n-gram با بیشترین احتمال انتخاب شده و token پایانی آن به عنوان کلمه خروجی داده می‌شود. به همین ترتیب token تولید می‌شود تا به __EOM__ برخورد کنیم.<br>این مدل unigram را نیز اجرا می‌کند و بدین صورت عمل می‌کند که به تعداد کلمات معمول یک مصراع شاهنامه، یعنی 6، متناسب با احتمال هر unigram آنها را تولید می‌کند. واضحاً چون با این روش هر کلمه مستقل از دیگری تولید می‌شود، نتیجه قابل قبول نیست.<br>یک boolean به عنوان ورودی تابع generate_mesra برای رعایت یا عدم رعایت قافیه در نظر گرفته می‌شود. متناسب با مقدار این boolean پس از ساخت مصراع دوم، کلمه‌ی آخر آن بررسی می‌گردد؛ به طور مثال اگر این boolean مقدار True داشت و دو مصراع هم‌قافیه نبودند، کلمه‌ی آخر مصراع ساخته شده به تعداد بار محدودی مجدداً ساخته می‌شود پس از هر بار ساخت مجدداً هم‌قافیه بودن بررسی می‌گردد. در صورتی‌که پس از این تعداد بار محدود هم‌چنان دو مصراع به شرط قافیه‌ی گفته شده نرسیده بودند، دیگر به این شرط توجهی نمی‌شود و شعر ساخته شده‌ی نهایی فارغ از هم‌قافیه بودن یا نبودن خروجی داده می‌شود.<br>بررسی هم‌قافیه بودن با مقایسه‌ی 2 حرف آخر دو کلمه انجام می‌گیرد. اگر دو حرف آخر دو کلمه یکسان بودند هم‌قافیه‌اند.
</div>

In [ ]:
class Ngram(object):
    
    UNK = '__UNK__'
    BOM = '__BOM__'
    EOM = '__EOM__'
    
    def __init__(self, data, n, laplace=1):
        self.vocab = dict()
        self.n = n
        self.laplace = laplace
        self.tokens = self.preprocess(data)
        self.vocab = nltk.FreqDist(self.tokens)
        self.model = self.create_model()
    
    def preprocess(self, data):
        tokens = data.strip().split()
        if not len(self.vocab):
            self.vocab = nltk.FreqDist(tokens)
        return [token if self.vocab[token] > 1 else Ngram.UNK for token in tokens]
    
    def create_model(self):
        num_tokens = len(self.tokens)
        if self.n == 1:
            model = { (unigram,): count / num_tokens for unigram, count in self.vocab.items() }
        else:
            model = self.smooth()
        return dict(sorted(model.items(), key=lambda x: x[1], reverse=True))
    
    def smooth(self):
        vocab_len = len(self.vocab)
        n_grams = nltk.ngrams(self.tokens, self.n)
        n_vocab = nltk.FreqDist(n_grams)
        m_grams = nltk.ngrams(self.tokens, self.n - 1)
        m_vocab = nltk.FreqDist(m_grams)
        
        def counted_smoothing(n_gram, n_count):
            m_gram = n_gram[:-1]
            m_count = m_vocab[m_gram]
            return (n_count + self.laplace) / (m_count + self.laplace * vocab_len)
        
        return { n_gram: counted_smoothing(n_gram, count) for n_gram, count in n_vocab.items() }
    
    def check_ghafie(self, ghafie1, ghafie2):
        margin = 3
        ham_ghafie = False
        if len(ghafie1) > 1 and len(ghafie2) > 1:
            for i in range(-1, -margin-1, -1):
                if ghafie1[i] != ghafie2[i]:
                    break
                else:
                    ham_ghafie = True
        return ham_ghafie
        
    
    def get_candidates(self, beginning_of_ngram, last_word, ghafie, ghafie_repeat, mesra1_ghafie):
        counter = 0
        best = None
        for ng in self.model.items():
            for i in range(self.n - 1):
                if ng[0][i] != beginning_of_ngram[i]:
                    break
            else:
                if ng[0][-1] != self.BOM and ng[0][-1] != self.UNK:
                    if not last_word:
                        return ng[0][-1]
                    if counter == 0:
                        best = ng[0][-1]
                    if ((ghafie and not self.check_ghafie(mesra1_ghafie, ng[0][-1])) or (not ghafie and self.check_ghafie(mesra1_ghafie, ng[0][-1]))) and counter < ghafie_repeat:
                        counter += 1
                    else:
                        if counter < ghafie_repeat:
                            return ng[0][-1]
        if best:
            return best
        return self.UNK
    
    def generate_mesra(self, first_mesra, ghafie):
        first_mesra = self.BOM + ' ' + first_mesra + ' ' + self.BOM
        first_mesra_tokens = self.preprocess(first_mesra)
        next_mesra = []
        ghafie_repeat = 5
        mesra1_ghafie = first_mesra_tokens[-2]
                
        if self.n == 1:
            vocab_list = [x[0] for x in self.model]
            vocab_probs = list(self.model.values())
            next_mesra = list(np.random.choice(vocab_list, p=vocab_probs, size=8, replace=False))
            
            if ghafie:
                if not self.check_ghafie(first_mesra_tokens[-2], next_mesra[-1]):
                    for i in range(ghafie_repeat):
                        next_mesra[-1] = np.random.choice(vocab_list, p=vocab_probs)
                        if self.check_ghafie(first_mesra_tokens[-2], next_mesra[-1]):
                            break
            else:
                if self.check_ghafie(first_mesra_tokens[-2], next_mesra[-1]):
                    for i in range(ghafie_repeat):
                        next_mesra[-1] = np.random.choice(vocab_list, p=vocab_probs)
                        if not self.check_ghafie(first_mesra_tokens[-2], next_mesra[-1]):
                            break
                
        
        else:
            beginning_of_ngram = tuple(first_mesra_tokens[-self.n + 1:])
            last_word = False
            while True:
                next_word = self.get_candidates(beginning_of_ngram, last_word, ghafie, ghafie_repeat, mesra1_ghafie)
                
                if next_word == self.EOM:
                    last_word = True
                    next_mesra[-1] = self.get_candidates(temp_beginning_of_ngram, last_word, ghafie, ghafie_repeat, mesra1_ghafie)
                    break
        
                next_mesra.append(next_word)
                temp_beginning_of_ngram = beginning_of_ngram
                beginning_of_ngram = (*beginning_of_ngram[1:], next_word)
                
        while self.EOM in next_mesra:
            next_mesra.remove(self.EOM)
        while self.BOM in next_mesra:
            next_mesra.remove(self.BOM)
        while self.UNK in next_mesra:
            next_mesra.remove(self.UNK)
        return(' '.join(next_mesra))

In [ ]:
gram5 = Ngram(beyt_file, 5)
gram4 = Ngram(beyt_file, 4)
gram3 = Ngram(beyt_file, 3)

In [ ]:
mesra1 = 'به گرد اندر آرد بهنگام کار'
gram4.generate_mesra(mesra1, False)

'ز دینار وز گوهر نابسود'

In [ ]:
gram4.generate_mesra(mesra1, True)

'ز دینار وز گوهر شاهوار'

In [ ]:
mesra1 = 'سپاه اندر آمد به پیش سوار'
gram4.generate_mesra(mesra1, True)

'سپردار و جوشنوران صد هزار'

In [ ]:
gram4.generate_mesra(mesra1, False)

'سپردار و جوشنوران صد هزار'

In [ ]:
mesra1 = 'بران تیغ زهر آب داده به دست'
gram5.generate_mesra(mesra1, True)

'همی رفت پویان به کردار مست'

In [ ]:
mesra1 = 'هاتفی از گوشه میخانه دوش'
gram5.generate_mesra(mesra1, True)

'گفت ببخشند گنه می بنوش'

In [ ]:
mesra1 = 'هزار دشمنم ار می کنند قصد هلاک'
gram3.generate_mesra(mesra1, False)

'ز هر سو که بد مهتری با گهر'

In [ ]:
mesra1 = 'سلامی چو بوی خوش آشنایی'
gram3.generate_mesra(mesra1, False)

'بدان تا شود نزد سالار توران سپاه'

<div dir="rtl"  style ='font-family: "B Nazanin";'>
    <h3> 3. مدل بر پایه LSTM </h3>
    ابتدا بررسی می‌کنیم که سیستم قابلیت استفاده از cuda را دارد یا نه.
</div>

In [ ]:
! pip install -q wandb
! git clone "https://github.com/amnghd/Persian_poems_corpus.git"
! mkdir "corpus"
! cp "Persian_poems_corpus/normalized/ferdousi_norm.txt" "Persian_poems_corpus/normalized/hafez_norm.txt" "Persian_poems_corpus/normalized/moulavi_norm.txt"
"./corpus/"


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# for accessing the files in google drive from google colab
from google.colab import drive
drive.mount('/content/drive')

Cloning into 'Persian_poems_corpus'...
remote: Enumerating objects: 159, done.
remote: Total 159 (delta 0), reused 0 (delta 0), pack-reused 159
Receiving objects: 100% (159/159), 45.21 MiB | 19.40 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Checking out files: 100% (148/148), done.
cp: target 'Persian_poems_corpus/normalized/moulavi_norm.txt' is not a directory
cpu
Mounted at /content/drive


<div dir="rtl"  style ='font-family: "B Nazanin";'>
    برای مانیتور کردن پارامترها و تغییرات مدل هنگام train شدن می‌توانیم از wandb استفاده کنیم. هم‌چنین امکان ذخیره‌ی پارامترها برای استفاده‌ی مجدد نیز وجود دارد. 
</div>

In [ ]:
wandb.init(project="ferdousi-generator")

wandb: Currently logged in as: thedrna (nlp400). Use `wandb login --relogin` to force relogin


In [ ]:
class Config:
    pass


config = Config()
# wandb.init(project="ferdousi-generator", name='all_poem_generic_tune')
# config = wandb.config
config.max_epochs = 50
config.batch_size = 256
config.embedding_size = 512
config.lstm_num_layers = 3
config.lstm_hidden_size = 512
config.sequence_length = 10
config.log_interval = 10
config.learning_rate = 0.001
config.vocab_size = 40000
config.lstm_dropout = 0.2

<div dir="rtl"  style ='font-family: "B Nazanin";'>
    در کلاس PoemDataset به خواندن و ایندکس کردن دیتا می‌پردازیم. هم‌چنین کلمات یکتای دیتا برای تشکیل vocabulary استخراج می‌گردند. برای هر کلمه‌ی یکتا، ایندکسی در نظر گرفته می‌شود و توسط تابع __getitem__ داده‌ای مناسب ورودی دادن به مدل گرفته می‌شود. از index_to_word و word_to_index در انکد و دیکد کردن استفاده می‌شود.<br>به کمک این کلاس می‌توان از میان داده‌های موجود، ابتدا به کمک تمامی اشعار  مجموعه‌ی واژگان کاملی ساخت و مدل را به کمک آن train کرد، سپس بر حسب هر شاعر آن را fine tune کرد. هم‌چنین برای پیش‌پردازش اشعار، ابتدا و انتهای مصراع‌ها و ابیات به همراه نام شاعر مشخص می‌گردد.
</div>

In [ ]:
from collections import Counter
    
class PoemDataset(torch.utils.data.Dataset):
    def __init__(
            self,
            config,
            device=torch.device('cpu'),
            poet='ferdousi',
            corpus_dir='./Persian_poems_corpus/normalized',
            vocab_path='./vocabulary.txt'
    ):
        self.config = config
        self.device = device

        self.words_by_poet = self.load_words(corpus_dir)
        self.poet = poet

    def preprocess_lines(self, lines, mask_key):
        lines = map(
            lambda i, line:
            f'[BOM_{mask_key}] ' + line + f' [EOS_{mask_key}]' if i % 2 == 0
            else f'[BOM_{mask_key}] ' + line,
            enumerate(lines)
        )
        lines = map(lambda line: line.split(' '), lines)
        words = itertools.chain.from_iterable(lines)
        return words

    def load_words(self, corpus_dir):
        words_by_poet = {}
        for filename in os.listdir(corpus_dir):
            with open(os.path.join(corpus_dir, filename)) as f:
                poet_name = filename.split('_')[0]
                lines = f.readlines()
                words_by_poet[poet_name] = self.preprocess_lines(lines, poet_name)
        return words_by_poet

    def load_vocabulary(self):
        pass

    @property
    def all_poets(self):
        return self.words_by_poet.keys()

    @property
    def poet(self):
        return self._poet

    @poet.setter
    def poet(self, poet):
        self._poet = poet
        if poet == 'all':
            self.words = itertools.chain.from_iterable(self.words_by_poet.values())
        else:
            self.words = self.words_by_poet[poet]
        self.index_to_word = {index: word for index, word in enumerate(self.vocabulary())}
        self.word_to_index = {word: index for index, word in enumerate(self.vocabulary())}

        self.words_indexes = [self.word_to_index[w] for w in self.words]

    def vocabulary(self):
        all_words = itertools.chain.from_iterable(self.words_by_poet.values())
        word_counts = Counter(all_words)
        vocabulary = sorted(word_counts, key=word_counts.get, reverse=True)
        return vocabulary

    def __len__(self):
        return len(self.words_indexes) - self.config.sequence_length

    def __getitem__(self, index):
        tensors = (
            torch.tensor(self.words_indexes[index:index + self.args.sequence_length]).to(self.device),
            torch.tensor(self.words_indexes[index + 1:index + self.args.sequence_length + 1]).to(self.device),
        )
        return tensors

<div dir="rtl"  style ='font-family: "B Nazanin";'>
    در کلاس Model مدل این بخش ساخته می‌شود که شامل سه بخش embedding، lstm و یک تابع خطی برای تبدیل خروجی lstm به vocab است. لایه‌های هر بخش توسط pytorch ساخته می‌شوند و مقادیر ابتدایی برابر با صفر است.
</div>

In [ ]:
from torch import nn

class Model(nn.Module):
    def __init__(self, dataset, config, device=torch.device('cpu')):
        super(Model, self).__init__()
        self.lstm_size = config.embedding_size
        self.lstm_hidden_size = config.lstm_hidden_size
        self.lstm_dropout = 0.2
        self.embedding_dim = config.embedding_size
        self.num_layers = config.lstm_num_layers
        self.device = device
        self.vocab_size = config.vocab_size
        self.embedding = nn.Embedding(
            num_embeddings=self.vocab_size,
            embedding_dim=self.embedding_dim,
        )
        self.lstm = nn.LSTM(
            input_size=self.lstm_size,
            hidden_size=self.lstm_hidden_size,
            num_layers=self.num_layers,
            dropout=self.lstm_dropout,
        )
        self.fc = nn.Linear(self.lstm_size, self.vocab_size)
        self.to(device)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.fc(output)
        return logits, state

    def init_state(self, sequence_length):
        return (torch.zeros(self.num_layers, sequence_length, self.lstm_size).to(self.device),
                torch.zeros(self.num_layers, sequence_length, self.lstm_size).to(self.device))

<div dir="rtl"  style ='font-family: "B Nazanin";'>
    نهایتاً یک تابع کلاسیک برای train داده‌ها استفاده می‌شود. در این تابع از optimizer جنریک Adam استفاده شده و loss به روش cross entropy محاسبه شده است. 
</div>

In [ ]:
from torch.utils.data import DataLoader

def train(dataset, model, args, checkpoint_path='/content/drive/MyDrive/NLP Class/checkpoints'):
    wandb.watch(model)
    model.train()

    dataloader = DataLoader(dataset, batch_size=args.batch_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)

    print({'batch_count': len(dataloader), 'epoch_count': args.max_epochs})
    for epoch in range(args.max_epochs):
        state_h, state_c = model.init_state(args.sequence_length)
        for batch, (x, y) in enumerate(dataloader):
            optimizer.zero_grad()

            y_pred, (state_h, state_c) = model(x, (state_h, state_c))
            loss = criterion(y_pred.transpose(1, 2), y)

            state_h = state_h.detach()
            state_c = state_c.detach()

            loss.backward()
            optimizer.step()
            print({'epoch': epoch, 'batch': batch, 'loss': loss.item()})
            if batch % args.log_interval == 0:
                wandb.log({"loss": loss})
        try:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
            }, os.path.join(checkpoint_path, f'model_{args.name}_checkpoint_{time.time()}.pt'))
        except:
            pass

<div dir="rtl"  style ='font-family: "B Nazanin";'>
    از تابع predict برای حدس ادامه‌ی بیت استفاده می‌گردد.<br> دیکودر مدل در این بخش قرار دارد، بطوریکه وزن‌های لازم برای انتخاب کلمه‌ی بعد از اعمال softmax روی hidden state آخرین مرحله حاصل می‌گردند. 
</div>

In [ ]:
def predict(dataset, model, text, next_words=100):
    model.eval()

    words = text.split(' ')
    state_h, state_c = model.init_state(len(words))

    for i in range(0, next_words):
        x = torch.tensor([[dataset.word_to_index[w] for w in words[i:]]]).to(device)
        y_pred, (state_h, state_c) = model(x, (state_h, state_c))

        last_word_logits = y_pred[0][-1]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().cpu().numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(dataset.index_to_word[word_index])

    return words

In [ ]:
dataset = PoemDataset(config, device)
# get first 10 items in dataset
for i in range(10):
    print(dataset[i])

FileNotFoundError: ignored

In [ ]:
model = Model(dataset)

train(dataset, model, config)

{'batch_count': 2609, 'epoch_count': 20}
{'epoch': 0, 'batch': 0, 'loss': 9.774300575256348}
{'epoch': 0, 'batch': 1, 'loss': 9.761186599731445}
{'epoch': 0, 'batch': 2, 'loss': 9.747979164123535}
{'epoch': 0, 'batch': 3, 'loss': 9.715853691101074}
{'epoch': 0, 'batch': 4, 'loss': 9.661108016967773}
{'epoch': 0, 'batch': 5, 'loss': 9.51749324798584}
{'epoch': 0, 'batch': 6, 'loss': 9.162014961242676}
{'epoch': 0, 'batch': 7, 'loss': 8.778281211853027}
{'epoch': 0, 'batch': 8, 'loss': 8.309832572937012}
{'epoch': 0, 'batch': 9, 'loss': 8.199350357055664}
{'epoch': 0, 'batch': 10, 'loss': 7.854308605194092}
{'epoch': 0, 'batch': 11, 'loss': 7.54002046585083}
{'epoch': 0, 'batch': 12, 'loss': 7.30495023727417}
{'epoch': 0, 'batch': 13, 'loss': 7.528113842010498}
{'epoch': 0, 'batch': 14, 'loss': 7.2615966796875}
{'epoch': 0, 'batch': 15, 'loss': 7.049915313720703}
{'epoch': 0, 'batch': 16, 'loss': 7.061543941497803}
{'epoch': 0, 'batch': 17, 'loss': 7.235649585723877}
{'epoch': 0, 'batch'

{'epoch': 0, 'batch': 154, 'loss': 6.281091690063477}
{'epoch': 0, 'batch': 155, 'loss': 6.36893892288208}
{'epoch': 0, 'batch': 156, 'loss': 6.665917873382568}
{'epoch': 0, 'batch': 157, 'loss': 6.727272033691406}
{'epoch': 0, 'batch': 158, 'loss': 6.911756992340088}
{'epoch': 0, 'batch': 159, 'loss': 6.5192646980285645}
{'epoch': 0, 'batch': 160, 'loss': 6.557824611663818}
{'epoch': 0, 'batch': 161, 'loss': 6.6095733642578125}
{'epoch': 0, 'batch': 162, 'loss': 6.348132610321045}
{'epoch': 0, 'batch': 163, 'loss': 6.350017547607422}
{'epoch': 0, 'batch': 164, 'loss': 6.460840225219727}
{'epoch': 0, 'batch': 165, 'loss': 6.307048320770264}
{'epoch': 0, 'batch': 166, 'loss': 6.697500705718994}
{'epoch': 0, 'batch': 167, 'loss': 6.321441173553467}
{'epoch': 0, 'batch': 168, 'loss': 6.460019588470459}
{'epoch': 0, 'batch': 169, 'loss': 7.023101329803467}
{'epoch': 0, 'batch': 170, 'loss': 6.68086051940918}
{'epoch': 0, 'batch': 171, 'loss': 6.909114837646484}
{'epoch': 0, 'batch': 172, '

{'epoch': 0, 'batch': 306, 'loss': 6.4696946144104}
{'epoch': 0, 'batch': 307, 'loss': 6.480348587036133}
{'epoch': 0, 'batch': 308, 'loss': 6.494039058685303}
{'epoch': 0, 'batch': 309, 'loss': 6.526309967041016}
{'epoch': 0, 'batch': 310, 'loss': 6.480602741241455}
{'epoch': 0, 'batch': 311, 'loss': 6.687238693237305}
{'epoch': 0, 'batch': 312, 'loss': 6.411061763763428}
{'epoch': 0, 'batch': 313, 'loss': 6.505953311920166}
{'epoch': 0, 'batch': 314, 'loss': 6.545412063598633}
{'epoch': 0, 'batch': 315, 'loss': 6.547349452972412}
{'epoch': 0, 'batch': 316, 'loss': 6.495586395263672}
{'epoch': 0, 'batch': 317, 'loss': 6.626834869384766}
{'epoch': 0, 'batch': 318, 'loss': 6.470386981964111}
{'epoch': 0, 'batch': 319, 'loss': 6.545322895050049}
{'epoch': 0, 'batch': 320, 'loss': 6.657440185546875}
{'epoch': 0, 'batch': 321, 'loss': 6.337181091308594}
{'epoch': 0, 'batch': 322, 'loss': 6.424967288970947}
{'epoch': 0, 'batch': 323, 'loss': 6.453615188598633}
{'epoch': 0, 'batch': 324, 'lo

{'epoch': 0, 'batch': 458, 'loss': 6.678541660308838}
{'epoch': 0, 'batch': 459, 'loss': 6.4600443840026855}
{'epoch': 0, 'batch': 460, 'loss': 6.379632949829102}
{'epoch': 0, 'batch': 461, 'loss': 5.9033589363098145}
{'epoch': 0, 'batch': 462, 'loss': 6.358068466186523}
{'epoch': 0, 'batch': 463, 'loss': 6.3226494789123535}
{'epoch': 0, 'batch': 464, 'loss': 6.641347885131836}
{'epoch': 0, 'batch': 465, 'loss': 6.4180989265441895}
{'epoch': 0, 'batch': 466, 'loss': 6.200371265411377}
{'epoch': 0, 'batch': 467, 'loss': 6.20464563369751}
{'epoch': 0, 'batch': 468, 'loss': 6.447671413421631}
{'epoch': 0, 'batch': 469, 'loss': 6.651864528656006}
{'epoch': 0, 'batch': 470, 'loss': 6.789651393890381}
{'epoch': 0, 'batch': 471, 'loss': 6.493434429168701}
{'epoch': 0, 'batch': 472, 'loss': 6.409309387207031}
{'epoch': 0, 'batch': 473, 'loss': 6.191265106201172}
{'epoch': 0, 'batch': 474, 'loss': 6.26267671585083}
{'epoch': 0, 'batch': 475, 'loss': 6.321670055389404}
{'epoch': 0, 'batch': 476,

{'epoch': 0, 'batch': 610, 'loss': 6.405157089233398}
{'epoch': 0, 'batch': 611, 'loss': 6.344775676727295}
{'epoch': 0, 'batch': 612, 'loss': 6.539079189300537}
{'epoch': 0, 'batch': 613, 'loss': 6.772803783416748}
{'epoch': 0, 'batch': 614, 'loss': 6.805748462677002}
{'epoch': 0, 'batch': 615, 'loss': 6.525693416595459}
{'epoch': 0, 'batch': 616, 'loss': 6.571883678436279}
{'epoch': 0, 'batch': 617, 'loss': 6.329531192779541}
{'epoch': 0, 'batch': 618, 'loss': 6.558544158935547}
{'epoch': 0, 'batch': 619, 'loss': 6.638702392578125}
{'epoch': 0, 'batch': 620, 'loss': 6.649852752685547}
{'epoch': 0, 'batch': 621, 'loss': 6.657663822174072}
{'epoch': 0, 'batch': 622, 'loss': 6.626964569091797}
{'epoch': 0, 'batch': 623, 'loss': 6.831180572509766}
{'epoch': 0, 'batch': 624, 'loss': 6.685838222503662}
{'epoch': 0, 'batch': 625, 'loss': 6.743122577667236}
{'epoch': 0, 'batch': 626, 'loss': 6.8544440269470215}
{'epoch': 0, 'batch': 627, 'loss': 6.563987731933594}
{'epoch': 0, 'batch': 628, 

...

{'epoch': 4, 'batch': 1453, 'loss': 4.475063800811768}
{'epoch': 4, 'batch': 1454, 'loss': 4.255180358886719}
{'epoch': 4, 'batch': 1455, 'loss': 4.378446102142334}
{'epoch': 4, 'batch': 1456, 'loss': 4.026584148406982}
{'epoch': 4, 'batch': 1457, 'loss': 4.229344844818115}
{'epoch': 4, 'batch': 1458, 'loss': 4.452343463897705}
{'epoch': 4, 'batch': 1459, 'loss': 4.233231544494629}
{'epoch': 4, 'batch': 1460, 'loss': 4.259684085845947}
{'epoch': 4, 'batch': 1461, 'loss': 4.353192329406738}
{'epoch': 4, 'batch': 1462, 'loss': 4.4191765785217285}
{'epoch': 4, 'batch': 1463, 'loss': 4.133487701416016}
{'epoch': 4, 'batch': 1464, 'loss': 4.277553558349609}
{'epoch': 4, 'batch': 1465, 'loss': 4.445662021636963}
{'epoch': 4, 'batch': 1466, 'loss': 4.215212345123291}
{'epoch': 4, 'batch': 1467, 'loss': 4.229988098144531}
{'epoch': 4, 'batch': 1468, 'loss': 4.414797306060791}
{'epoch': 4, 'batch': 1469, 'loss': 4.135349750518799}
{'epoch': 4, 'batch': 1470, 'loss': 4.301911354064941}
{'epoch':

{'epoch': 4, 'batch': 1602, 'loss': 4.050332069396973}
{'epoch': 4, 'batch': 1603, 'loss': 4.124492168426514}
{'epoch': 4, 'batch': 1604, 'loss': 4.389887809753418}
{'epoch': 4, 'batch': 1605, 'loss': 4.559385776519775}
{'epoch': 4, 'batch': 1606, 'loss': 4.328335285186768}
{'epoch': 4, 'batch': 1607, 'loss': 4.185894966125488}
{'epoch': 4, 'batch': 1608, 'loss': 4.151832103729248}
{'epoch': 4, 'batch': 1609, 'loss': 4.212386608123779}
{'epoch': 4, 'batch': 1610, 'loss': 4.245310306549072}
{'epoch': 4, 'batch': 1611, 'loss': 3.88130259513855}
{'epoch': 4, 'batch': 1612, 'loss': 4.0442352294921875}
{'epoch': 4, 'batch': 1613, 'loss': 3.7711451053619385}
{'epoch': 4, 'batch': 1614, 'loss': 4.51833963394165}
{'epoch': 4, 'batch': 1615, 'loss': 4.17262601852417}
{'epoch': 4, 'batch': 1616, 'loss': 3.973562002182007}
{'epoch': 4, 'batch': 1617, 'loss': 4.058842182159424}
{'epoch': 4, 'batch': 1618, 'loss': 3.8785431385040283}
{'epoch': 4, 'batch': 1619, 'loss': 4.391660690307617}
{'epoch': 

{'epoch': 4, 'batch': 1751, 'loss': 4.227108955383301}
{'epoch': 4, 'batch': 1752, 'loss': 4.13158655166626}
{'epoch': 4, 'batch': 1753, 'loss': 4.526743412017822}
{'epoch': 4, 'batch': 1754, 'loss': 4.18266487121582}
{'epoch': 4, 'batch': 1755, 'loss': 4.29818058013916}
{'epoch': 4, 'batch': 1756, 'loss': 3.985320806503296}
{'epoch': 4, 'batch': 1757, 'loss': 4.342442035675049}
{'epoch': 4, 'batch': 1758, 'loss': 4.49359655380249}
{'epoch': 4, 'batch': 1759, 'loss': 3.931694984436035}
{'epoch': 4, 'batch': 1760, 'loss': 4.335658073425293}
{'epoch': 4, 'batch': 1761, 'loss': 4.595249652862549}
{'epoch': 4, 'batch': 1762, 'loss': 4.288554668426514}
{'epoch': 4, 'batch': 1763, 'loss': 4.254398822784424}
{'epoch': 4, 'batch': 1764, 'loss': 4.361176490783691}
{'epoch': 4, 'batch': 1765, 'loss': 4.197258472442627}
{'epoch': 4, 'batch': 1766, 'loss': 4.252795696258545}
{'epoch': 4, 'batch': 1767, 'loss': 4.286163806915283}
{'epoch': 4, 'batch': 1768, 'loss': 4.2218098640441895}
{'epoch': 4, 

{'epoch': 4, 'batch': 1900, 'loss': 3.9309804439544678}
{'epoch': 4, 'batch': 1901, 'loss': 4.363471508026123}
{'epoch': 4, 'batch': 1902, 'loss': 4.481727123260498}
{'epoch': 4, 'batch': 1903, 'loss': 4.588984966278076}
{'epoch': 4, 'batch': 1904, 'loss': 4.200727462768555}
{'epoch': 4, 'batch': 1905, 'loss': 4.330854892730713}
{'epoch': 4, 'batch': 1906, 'loss': 4.399538516998291}
{'epoch': 4, 'batch': 1907, 'loss': 3.994715452194214}
{'epoch': 4, 'batch': 1908, 'loss': 4.568429946899414}
{'epoch': 4, 'batch': 1909, 'loss': 4.48027229309082}
{'epoch': 4, 'batch': 1910, 'loss': 4.312780857086182}
{'epoch': 4, 'batch': 1911, 'loss': 4.052121162414551}
{'epoch': 4, 'batch': 1912, 'loss': 4.220328330993652}
{'epoch': 4, 'batch': 1913, 'loss': 4.339457988739014}
{'epoch': 4, 'batch': 1914, 'loss': 4.244417667388916}
{'epoch': 4, 'batch': 1915, 'loss': 4.099876403808594}
{'epoch': 4, 'batch': 1916, 'loss': 4.216757774353027}
{'epoch': 4, 'batch': 1917, 'loss': 4.135529041290283}
{'epoch': 

KeyboardInterrupt: 

In [ ]:
print('\n'.join(predict(dataset, model, text='[BOM] توانا بود هر که')))

In [ ]:
# save torch model and configs
import time

torch.save({'model_state_dict': model.state_dict()}, f'../data/checkpoints/model_{time.time()}.pt')
wandb.finish()


<div dir="rtl"  style ='font-family: "B Nazanin";'>
    <h3> 3. مدل بر پایه مکانیسم توجه </h3>
    ابتدا به کمک کلاس Cleaner پیش‌پردازش‌های لازم را انجام می‌دهیم که شامل خواندن متن و افزودن عبارات اول و پایان مصراع‌ها می‌باشد.
</div>

In [ ]:
class Cleaner:
    def __init__(self):
        self.normalizer = Normalizer()
        # self.stop_words = [self.normalizer.normalize(x.strip()) for x in codecs.open('data/stopwords.txt','r','utf-8').readlines()]
        self.file_range = None

    def add_statement_mesras(self, file_in= "./data/ferdousi_norm.txt", file_out='./data/f_beyt.txt'):
        start_mesra = '[BOM]'
        end_mesra = ''
        start_beyt = ''
        end_beyt = '[EOS]' + os.linesep
        beyt_file = ''
        with open(file_in, 'r', encoding="utf-8") as fp:
            lines = fp.readlines()
            for i in range(0, len(lines) - 1, 2):
                mesra1 = start_mesra + lines[i].strip() + end_mesra
                mesra2 = start_mesra + lines[i + 1].strip() + end_mesra
                b = start_beyt + mesra1.strip() + ' ' + mesra2.strip() + end_beyt
                beyt_file += b
                
        with open(file_out, 'w', encoding="utf-8") as fp:
            fp.write(beyt_file)

    def read_poems(self, file_patterns: str, file_range: tuple, normalize=False):  #NOT USED
        file_range = range(*file_range) if file_range is not None else None 
        file_names = glob.glob(file_patterns)
        mesra_collection = []
        for file_name in file_names:
            if normalize:
                mesra_collection += [self.normalizer.normalize(x) for x in codecs.open(file_name,'rU','utf-8').readlines()]
            else:
                mesra_collection += codecs.open(file_name,'rU','utf-8').readlines()[2:]
        return mesra_collection
    

    def read_documents(self): #NOT USED
        poems = self.read_poems(self.file_pattern, self.file_range)
        cleaned = self.clean_data(poems)
        return ' \n '.join(cleaned)

<div dir="rtl"  style ='font-family: "B Nazanin";'>
     برای ساخت مدل بر پایه مکانیسم توجه، از مدل GPT2 بر پایه <a href="https://huggingface.co/bolbolzaban/gpt2-persian">مدل pre_trained بلبل‌زبان</a> استفاده کردیم. توضیحات این مدل و دیتای مورد استفاده‌ی آن در <a href="https://khashei.medium.com/a-not-so-dangerous-ai-in-the-persian-language-39172a641c84">این لینک</a> قابل مشاهده است. این مدل را بر اساس داده‌های مورد استفاده‌ی خودمان fine tune کردیم.<br> ابتدا این مدل گرفته می‌شود و متغیرهای لایه‌های درونی آن freeze می‌گردند، سپس داده‌ی اشعار پیش‌پردازش شده tokenize می‌شود و به دو بخش train و test تقسیم می‌گردد. برای train کردن، مدل بلبل‌زبان را بر اساس این داده fine tune می‌کنیم.<br>خروجی این مدل یک generator است که برای ساخت مصراع دوم از pipeline استفاده می‌کند که ورودی (مصراع اول) را به ساختاری مناسب ورودی دادن به مدل (با پیش‌پردازش و encoding) تبدیل کرده و به کمک مدل train شده، مصراع دوم را ساخته و decode می‌کند و بدین ترتیب بیت را می‌سازد.<br>کیفیت خروجی به تعداد epochها و سایز batchها بستگی خواهد داشت که افزایش آنها باعث بهبود نتیجه و کاهش سرعت training (بسته به سخت‌افزار سیستم) می‌گردد. ما پس از چندین آزمون و خطا با در نظر گرفتن مقدار 10 برای این دو متغیر به نتایج نسبتاً مناسب در زمان مطلوب رسیدیم.<br>کد این بخش روی gpu اجرا شده و در صورت نبودن gpu در سیستم، بخش مربوط به device در آن تغییر می‌یابد. مدل نهایی این بخش در <a href="url">اینجا</a> قرار گرفته و قابل دانلود است.<br>برای اعمال رعایت یا عدم رعایت قافیه در این بخش، دو مدل مجزا از هم روی دو نمونه شعر با قافیه (شاهنامه) و بدون قافیه (غزلیات حافظ) train می‌شوند و با توجه به ورودی قافیه، نتیجه مدل مربوطه به عنوان خروجی نمایش داده می‌شود.
</div>

In [ ]:
class MesraGenerator:
    def __init__(self, train_path, test_path):
        self.base_path = "./data"
        self.model = None
        self.tokenizer = None
        self.generator = None
        self.train_path = os.path.join(self.base_path, train_path)
        self.test_path = os.path.join(self.base_path, test_path)
        # self.cleaner = Cleaner()

    def read_data(self, tokenizer, train_path=None, test_path=None):
        path = train_path if train_path is not None else self.train_path
        test_path = test_path if test_path is not None else self.test_path
        train_dataset = TextDataset(
            tokenizer=tokenizer, file_path=path, block_size=128)
        
        test_dataset = TextDataset(
            tokenizer=tokenizer, file_path=test_path, block_size=128)
        
        data_collator = DataCollatorForLanguageModeling(
            tokenizer=tokenizer,
            mlm=False,
        )
        # train_texts, val_texts = train_test_split(train_dataset)
        return train_dataset, test_dataset, data_collator

    def read_model(self, model_type='bolbolzaban/gpt2-persian'):
        self.model = GPT2LMHeadModel.from_pretrained(model_type)
        self.tokenizer = AutoTokenizer.from_pretrained('bolbolzaban/gpt2-persian')
        return self.model, self.tokenizer

    def freeze_lower_layers(self):
        for param in self.model.base_model.parameters():
            param.requires_grad = False

        for param in (
            self.model.base_model.h[23].parameters() or self.model.base_model.h[22].parameters()
        ):
            param.requires_grad = True
            
    def fine_tune_model(self, model, tokenizer, train_texts, val_texts, data_collator):
        training_args = TrainingArguments(
        output_dir="./model",
        overwrite_output_dir=True,
        num_train_epochs=5,
        # Set the batch size to a maximum value that could fit into GPU memory,
        # for example 12 is the largest batch size that could work on a 6gb GPU when training the last to layers
        per_device_train_batch_size=12,
        per_device_eval_batch_size=12,
        eval_steps=1000,
        save_steps=1000,
        warmup_steps=500)

        trainer = Trainer(
            model=model,
            args=training_args,
            data_collator=data_collator,
            train_dataset=train_texts,
            eval_dataset=val_texts,
        )
        trainer.train()
        return trainer
        
    def init_generator(self):      
        model, tokenizer = self.read_model()
        self.freeze_lower_layers()
        train_texts, val_texts, data_collator = self.read_data(tokenizer)
        trainer = self.fine_tune_model(model, tokenizer, train_texts, val_texts, data_collator)
        generator = pipeline('text-generation', trainer.model, tokenizer=tokenizer, config={'max_length':256}, device=0)
        self.generator = generator
        return generator

    def save_model(self, dir=None):
        self.trainer.save_model(output_dir=dir)

In [ ]:
cleaner = Cleaner()
cleaner.add_statement_mesras()

In [ ]:
train_path = "f_beyt.txt"
test_path = "test.txt"
mesra_model = MesraGenerator(train_path, test_path)
generator = mesra_model.init_generator()
# train_texts, val_texts, data_collator = generator.read_data()
generator('[BOM]به گرد اندر آرد بهنگام کار')

In [ ]:
list(np.random.choice([1,2,3,4], size=2))

[3, 4]